In [16]:
import nltk
import random
import feedparser

In [17]:
urls ={
        'mlb':'https://sports.yahoo.com/mlb/rss.xml',
        'nfl':'https://sports.yahoo.com/nfl/rss.xml'
}

In [18]:
feedmap = {}
stopwords = nltk.corpus.stopwords.words('english')

def featureExtractor(words):
    features ={}
    for word in words:
        if word not in stopwords:
            features["word({})".format(word)] = True
    return features

sentences = []

for category in urls.keys():
    feedmap[category] = feedparser.parse(urls[category])
    print("downloading {}".format(urls[category]))
    for entry in feedmap[category]['entries']:
        data = entry['summary']
        words = data.split()
        sentences.append((category,words))
        


downloading https://sports.yahoo.com/mlb/rss.xml
downloading https://sports.yahoo.com/nfl/rss.xml


In [19]:
featuresets = [(featureExtractor(words),category)for category,words in sentences]
random.shuffle(featuresets)



In [20]:
total = len(featuresets)
off = int(total/2)
trainset = featuresets[off:]
testset = featuresets[:off]

In [21]:
classifier = nltk.NaiveBayesClassifier.train(trainset)
print(nltk.classify.accuracy(classifier,testset))

0.8823529411764706


In [22]:
classifier.show_most_informative_features(5)

for (i,entry) in enumerate(feedmap['nfl']['entries']):
    if i<4:
        features = featureExtractor(entry['title'].split())
        category = classifier.classify(features)
        print(('{} -> {}'.format(category,entry['title'])))

Most Informative Features
           word(Angeles) = True              mlb : nfl    =      3.8 : 1.0
             word(first) = True              mlb : nfl    =      2.4 : 1.0
               word(Los) = True              mlb : nfl    =      2.3 : 1.0
               word(The) = True              mlb : nfl    =      2.0 : 1.0
            word(Braves) = None              nfl : mlb    =      1.8 : 1.0
nfl -> Forget Super Bowl, are Chiefs a playoff team?
mlb -> Matthew Stafford: It was cool to see Lions fans in L.A. wearing my jersey
nfl -> NFL Week 7 winners and losers: Tom Brady still playing at an elite level. The AFC is a parity party.
nfl -> Why Detroit Lions' D'Andre Swift is 'a stud, can't get him the ball enough'
